In [1]:
import pandas as pd
import numpy as np

In [17]:
def charge_df(path):
    df = pd.read_csv(path)
    df = df.drop(["Recomender", "RMSE", "Precision@10", "Recall@10"], axis=1)
    
    for col in df.columns[1:]:
        df[col] = df[col].apply(lambda row: float(row.replace(",", ".")))
        
    return df

In [18]:
def maut(df, 
        algorithms, 
        metrics, 
        error_metrics=['MSE', 'MAE']):
    
    utility_scores = {}
    
    for metric in metrics:
        scores = df[metric].values
        
        if metric in error_metrics: 
            score_min = max(scores)
            score_max = min(scores)
        else:
            score_min = min(scores)
            score_max = max(scores)
            
        for algorithm in algorithms:
            alg_score = df.loc[df['Algoritmo'].str.contains(algorithm)][metric].values[0]
            utility_score = (alg_score - score_min) / (score_max - score_min)
            
            if algorithm in utility_scores.keys():
                utility_scores[algorithm].append(utility_score)
            else:
                utility_scores[algorithm] = [utility_score]
    
    
    ranking = {}
    for alg, scores in utility_scores.items():
        
        efetivity = scores[:4]
        diversity = scores[4:]
        
        ranking[alg] = [round(np.array(efetivity).sum(), 2), round(np.array(diversity).sum(), 2)]
        
    return np.array(list(ranking.values()))

In [20]:
tampa = charge_df("tampa.csv")
tampa_values = maut(tampa, tampa['Algoritmo'].values, tampa.columns[1:])

In [21]:
phila = charge_df("phila.csv")
phila_values = maut(phila, phila['Algoritmo'].values, phila.columns[1:])

In [22]:
video = charge_df("video.csv")
video_values = maut(video, video['Algoritmo'].values, video.columns[1:])

In [23]:
maut_rank = tampa_values + phila_values + video_values
maut_rank

array([[4.75, 4.05],
       [8.11, 1.93],
       [2.5 , 3.58],
       [9.77, 1.79],
       [6.73, 1.28],
       [3.93, 3.44],
       [3.42, 4.05],
       [9.71, 1.51],
       [8.33, 1.42],
       [5.44, 2.41]])

In [26]:
efetivity_rank = [(algorithm, round(score, 2)) for algorithm, score in zip(video['Algoritmo'], maut_rank[:, 0])]
efetivity_rank

[('DeepCoNN', 4.75),
 ('D ATTN', 8.11),
 ('MPCN', 2.5),
 ('NARRE', 9.77),
 ('DAML', 6.73),
 ('CARL', 3.93),
 ('CARP', 3.42),
 ('ANR', 9.71),
 ('HRDR', 8.33),
 ('RGNN', 5.44)]

In [30]:
efetivity = pd.DataFrame(efetivity_rank, columns=['Algoritmo', 'Score'])
efetivity.sort_values(by="Score", ascending=False)

,Algoritmo,Score
3,NARRE,9.77
7,ANR,9.71
8,HRDR,8.33
1,D ATTN,8.11
4,DAML,6.73
9,RGNN,5.44
0,DeepCoNN,4.75
5,CARL,3.93
6,CARP,3.42
2,MPCN,2.50


In [31]:
diversity_rank = [(algorithm, round(score, 2)) for algorithm, score in zip(video['Algoritmo'], maut_rank[:, 1])]
diversity = pd.DataFrame(diversity_rank, columns=['Algoritmo', 'Score'])
diversity.sort_values(by="Score", ascending=False)

,Algoritmo,Score
0,DeepCoNN,4.05
6,CARP,4.05
2,MPCN,3.58
5,CARL,3.44
9,RGNN,2.41
1,D ATTN,1.93
3,NARRE,1.79
7,ANR,1.51
8,HRDR,1.42
4,DAML,1.28
